In [1]:
# Initialising the SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logistic_regression').getOrCreate()

In [2]:
# Loading the dataset
df = spark.sql('SELECT * FROM customer_churn_csv')

In [3]:
# Examining the dataset
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [4]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [5]:
# Transforming the dataframe into one accepted by PySpark
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'],
                           outputCol='features')
final_df = assembler.transform(df).select('features', 'Churn')

In [6]:
# Train test split
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [7]:
# Creating the logistic regression model
from pyspark.ml.classification import LogisticRegression
classifier = LogisticRegression(featuresCol='features', labelCol='Churn', predictionCol='prediction')
fitted_classifier = classifier.fit(train_data)

In [8]:
# Evaluate
summary = fitted_classifier.summary
summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| Churn| prediction|
+-------+-------------------+-------------------+
 count| 650| 650|
 mean|0.17846153846153845|0.13076923076923078|
 stddev|0.38319567297671353|0.33740744410179924|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [9]:
# Evaluating using the testset
pred_vs_actual = fitted_classifier.evaluate(test_data)
pred_vs_actual.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[28.0,11128.95,5....| 0|[4.58399700115357...|[0.98988928149904...| 0.0|
[29.0,8688.17,5.7...| 1|[2.99152067181309...|[0.95218958571625...| 0.0|
[29.0,9617.59,5.4...| 0|[4.28619817040240...|[0.98642956199762...| 0.0|
[29.0,11274.46,4....| 0|[4.96345458479928...|[0.99305976067093...| 0.0|
[29.0,13255.05,4....| 0|[4.62321563172067...|[0.99027435318872...| 0.0|
[30.0,7960.64,2.7...| 1|[3.73970665571636...|[0.97679041240472...| 0.0|
[30.0,8874.83,5.5...| 0|[3.00410978085719...|[0.95275944803909...| 0.0|
[30.0,10960.52,5....| 0|[2.70201189109760...|[0.93714525670415...| 0.0|
[30.0,12788.37,4....| 0|[2.59830344127175...|[0.93075231217610...| 0.0|
[31.0,5304.6,5.29...| 0|[3.16772837433120...|[0.95960161380966...| 0.0|
[31.0,7073.61,5.7...| 0|[2.84443590137212...|[0.94503035377843...| 0.0|
[31.0,8829.83,4.5...| 0|[4.79067190415669...|[0.99176156158734...| 0.0|
[31.0,10182.6,3.7...| 0|[5.26676546884944...|[0.99486621705273...| 0.0|
[32.0,8575.71,4.2...| 0|[3.71989396749836...|[0.97633697228995...| 0.0|
[32.0,8617.98,6.2...| 1|[1.25679217036040...|[0.77847340353714...| 0.0|
[32.0,10716.75,5....| 0|[4.28142371278905...|[0.98636550110965...| 0.0|
[32.0,12142.99,5....| 0|[5.50099900837661...|[0.99593390983053...| 0.0|
[32.0,13630.93,4....| 0|[2.38241547194090...|[0.91547652935743...| 0.0|
[33.0,7492.9,6.71...| 0|[4.37162558672043...|[0.98752685288833...| 0.0|
[33.0,7750.54,4.5...| 0|[4.62108494200457...|[0.99025381094008...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [10]:
# Area under the curve (ROC) evalutation
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
evaluator.evaluate(pred_vs_actual.predictions)

Out[44]: 0.8003812636165577

In [11]:
# Predicting using unlabeled data
# Loading the dataset
df_unlabeled = spark.sql('SELECT * FROM new_customers_csv')

In [12]:
df_unlabeled.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+

In [13]:
# Transforming the dataframe into one accepted by PySpark
final_df_unlabeled = assembler.transform(df_unlabeled)

In [14]:
final_df_unlabeled.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,7.7...|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,9.2...|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,9.4,...|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,10...|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,3.4...|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+

In [15]:
# Creating a new model using the entire dataset
classifier_all = classifier.fit(final_df)
results = classifier_all.transform(final_df_unlabeled)

In [16]:
results.select('Company', 'prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+